In [ ]:
# externals
import os
import pathlib
import platform
import re
import shutil
import subprocess

In [ ]:
# get the current working directory
cwd = pathlib.PosixPath(os.getcwd())
# show me
print(f"cwd: {cwd}")

In [ ]:
# the name of the app, hence the name of its environment
app = "qed"

# the current working directory
cwd = pathlib.PosixPath(os.getcwd())
# the directory with the support files
support_dir = cwd / "support"

# the user's home directory
home_dir = pathlib.PosixPath("~").expanduser()

# the home of local conda environments
base_dir = home_dir / ".local"
# the home of tools shared by all environments
base_bin_dir = base_dir / "bin"

# the location of the qed environment
qed_dir = base_dir / "envs" / app

# the location of the pyre configuration directory
pyre_dir = home_dir / ".pyre"
# if you want XDG compliance, use "~/.config/pyre"

# the location with the mm package database
mm_dir = home_dir / ".mm"
# if you want XDG compliance, use "~/.config/mm"

# the location of the source code
# we will clone the various repositories in here
# feel free to rename this to whatever makes sense to you
src_dir = home_dir / "dv"

# the install location of the build products
prefix_dir = qed_dir

# temporary storage for the intermediate build products
bldroot_dir = home_dir / "tmp/builds/mm"

In [ ]:
# create the directories we need
# the base environment directory
base_dir.mkdir(parents=True, exist_ok=True)
# and the place with the shared tool
base_bin_dir.mkdir(parents=True, exist_ok=True)
# the pyre configuration directory
pyre_dir.mkdir(parents=True, exist_ok=True)
# the location of the mm package database
mm_dir.mkdir(parents=True, exist_ok=True)
# finaly, the source directory
src_dir.mkdir(parents=True, exist_ok=True)

# a table that maps system names to the ones used by the micromamba server
systems = {
    "Darwin": "osx",
    "Linux": "linux",
}
# and one that maps the architectures
machines = {
    "arm64": "arm64",
    "ppc64le": "ppc64le",
    "x86_64": "64",
}
# interrogate the host
uname = platform.uname()
# extract what we need
system = systems[uname.system]
machine = machines[uname.machine]
# build the url
url = f"https://micro.mamba.pm/api/micromamba/{system}-{machine}/latest"

# fetch micromamba and install it
os.system(f"curl -Ls {url} | tar xvj --directory {base_dir} bin/micromamba")

# alias
micromamba = base_bin_dir / "micromamba"

# mark
os.environ["MAMBA_ROOT_PREFIX"] = str(base_dir)

In [ ]:
# micromamba fails if the environment directory exists
if qed_dir.exists():
    # so we may need to remove it
    shutil.rmtree(qed_dir)

# the environment configuration file is machine specific
env_file = support_dir / f"env-{system}-{machine}.yaml"

# install the environment
os.system(f"{micromamba} create --yes --prefix {qed_dir} --file {support_dir / env_file}")

In [ ]:
# build a shortcut for running commands in this environment
run = f"{micromamba} run --prefix {qed_dir}"

In [ ]:
# find the version of python
# there is no easy way do this...

# set up the command line
argv = [
    micromamba,
    "run",
    "--prefix", str(qed_dir),
    "python3", "--version"
]
# and the options to popen
options = {
    "executable": micromamba,
    "args": argv,
    "stdout": subprocess.PIPE,
    "stderr": subprocess.PIPE,
    "universal_newlines": True,
    "shell": False,
}
# launch the interpreter
with subprocess.Popen(**options) as cmd:
    # collect the output
    stdout, stderror = cmd.communicate()
    # get the status
    status = cmd.returncode
    # if something went wrong
    if status != 0:
        # ooops
        raise RuntimeError("could not retrieve the version of the python interpreter")
    # parse the output
    match = re.match(r"Python (?P<major>\d+)\.(?P<minor>\d+).(?P<micro>\d+)", stdout)
    # if we failed to match
    if not match:
        # ooops
        raise RuntimeError("could not retrieve the version of the python interpreter")
    # unpack
    python_major = match.group("major")
    python_minor = match.group("minor")
    python_micro = match.group("micro")

    # and show me
    print(f"python version: {python_major}.{python_minor}.{python_micro}")

In [ ]:
# build the source directory
src_dir.mkdir(parents=True, exist_ok=True)

In [ ]:
# go to the source directory
os.chdir(src_dir)
# clone or update mm
mm_src_dir = src_dir / "mm"

# cloning will fail if the directory exists
if mm_src_dir.exists():
    # so update it instead: go there
    os.chdir(mm_src_dir)
    # pull
    os.system(f"{run} git pull")
    # and go back
    os.chdir(src_dir)
# if the directory doesn't exist
else:
    # clone it
    os.system(f"{run} git clone https://github.com/aivazis/mm")

# the mm configuration file
mm_in_file = support_dir / f"mm-{system}-{machine}.yaml"
# gets installed here
mm_out_file = pyre_dir / "mm.yaml"
# copy the file
shutil.copy(mm_in_file, mm_out_file)

# the mm package database
pkg_in_file = support_dir / "config.mm"
# gets installed here
pkg_out_file = mm_dir / "config.mm"
# open the output file
with open(pkg_out_file, mode="w") as ostream:
    # and the input file
    with open(pkg_in_file, mode="r") as istream:
        # read the contents
        contents = istream.read()
        # expand the macros
        contents = contents.replace("@PYTHON_VERSION@", f"{python_major}.{python_minor}")
        # and write them out
        ostream.write(contents)

# setup the command for invoking mm
mm = f"{run} python3 {mm_src_dir / 'mm'}"

In [ ]:
# go to the source directory
os.chdir(src_dir)
# clone or update pyre
pyre_src_dir = src_dir / "pyre"
# cloning will fail if the directory exists
if pyre_src_dir.exists():
    # so update it instead: go there
    os.chdir(pyre_src_dir)
    # pull
    os.system(f"{run} git pull")
# if the directory doesn't exist
else:
    # clone it
    os.system(f"{run} git clone https://github.com/pyre/pyre")
    # and go there
    os.chdir(pyre_src_dir)

# show me where it will end up
os.system(f"{mm} builder.info")
# build it
os.system(f"{mm}")

In [ ]:
# next, qed
os.chdir(src_dir)
# clone or update pyre
qed_src_dir = src_dir / "qed"
# cloning will fail if the directory exists
if qed_src_dir.exists():
    # so update it instead: go there
    os.chdir(qed_src_dir)
    # pull
    os.system(f"{run} git pull")
# if the directory doesn't exist
else:
    # clone it
    os.system(f"{run} git clone https://github.com/aivazis/qed")
    # and go there
    os.chdir(qed_src_dir)

# show me where it will end up
os.system(f"{mm} builder.info")
# build it
os.system(f"{mm}")

In [ ]:

# finally, the hdf5 repository with the necessary bug fixes
# we'll remove this once there a release that works
h5_src_dir = src_dir / "qed"
# cloning will fail if the directory exists
if h5_src_dir.exists():
    # so we may have to remove it
    shutil.rmtree(h5_src_dir)
# clone
os.system(f"{run} git clone https://github.com/aivazis/hdf5")

In [ ]:
# build pyre

# go to its source directory
os.chdir(pyre_src_dir)

# build it
os.system(f"{run} python3 {mm_src_dir / 'mm.py'}")